# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770901414085         NaN   2.99e-01   0.80    8.0
  2   -2.772142295214   -1.24e-03   4.82e-02   0.80    1.0
  3   -2.772170216654   -2.79e-05   2.95e-03   0.80    1.0
  4   -2.772170695422   -4.79e-07   4.20e-04   0.80    2.0
  5   -2.772170721638   -2.62e-08   1.18e-04   0.80    2.0
  6   -2.772170723014   -1.38e-09   6.02e-06   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770767737517         NaN   2.99e-01   0.80    9.0
  2   -2.772060798991   -1.29e-03   4.75e-02   0.80    1.0
  3   -2.772082492245   -2.17e-05   4.46e-03   0.80    2.0
  4   -2.772083417591   -9.25e-07   1.15e-04   0.80    2.0
  5   -2.772083417802   -2.11e-10   1.76e-05   0.80    2.0


1.7735832738990125

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770762436678         NaN   2.99e-01   0.80    9.0
  2   -2.772059901562   -1.30e-03   4.78e-02   0.80    1.0
  3   -2.772082756136   -2.29e-05   4.06e-03   0.80    2.0
  4   -2.772083417556   -6.61e-07   1.02e-04   0.80    2.0
  5   -2.772083417804   -2.48e-10   1.49e-05   0.80    2.0

Polarizability via ForwardDiff:       1.772544502796599
Polarizability via finite difference: 1.7735832738990125
